# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Estudo de Caso 3</font>
## <font color='blue'>Construindo e Treinando Um Modelo de Linguagem Transformer a Partir do Zero em Português</font>

![DSA](imagens/EC3.png)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.9


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
import random
random.seed(10)

In [4]:
# Imports
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

re   : 2.2.1
torch: 2.0.1
numpy: 1.23.5



## Carregando os Dados de Texto

In [6]:
# Carrega os dados de texto
text = open('dados/frases.txt', 'r').read()

In [7]:
type(text)

str

In [8]:
print(text)

'Olá, como vai? Eu sou a Ana.\n'
'Olá, Ana, meu nome é Carlos. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau Parabéns, Carlos!\n'
'Obrigado Ana.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Ana?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'


## Pré-Processamento dos Dados de Texto e Construção do Vocabulário

In [9]:
# Filtramos caracteres especiais: '.', ',', '?', '!'
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n') 

In [10]:
print(sentences)

["'olá como vai eu sou a ana\\n'", "'olá ana meu nome é carlos muito prazer\\n'", "'prazer em conhecer você também como você está hoje\\n'", "'ótimo meu time de futebol venceu a competição\\n'", "'uau parabéns carlos\\n'", "'obrigado ana\\n'", "'vamos comer uma pizza mais tarde para celebrar\\n'", "'claro você recomenda algum restaurante ana\\n'", "'sim abriu um restaurante novo e dizem que a pizza de banana é fenomenal\\n'", "'ok nos encontramos no restaurante às sete da noite pode ser\\n'", "'pode sim nos vemos mais tarde então'"]


In [11]:
# Dividimos as frases em palavras e criamos uma lista de palavras
word_list = list(set(" ".join(sentences).split()))

In [12]:
print(word_list)

["'pode", 'uma', "então'", 'abriu', 'nos', 'também', "ser\\n'", 'banana', 'está', 'time', 'conhecer', "celebrar\\n'", 'restaurante', 'sim', "'prazer", "'ótimo", 'sou', 'meu', 'de', 'pizza', "'claro", 'recomenda', "'sim", 'é', "'uau", 'sete', 'futebol', 'tarde', 'em', "hoje\\n'", 'da', 'venceu', 'vemos', "'ok", "ana\\n'", 'no', 'muito', 'comer', 'mais', 'vai', 'novo', 'que', 'carlos', 'a', 'um', 'encontramos', 'você', 'algum', 'parabéns', 'e', "'olá", 'para', 'dizem', 'noite', 'pode', "carlos\\n'", "'obrigado", "fenomenal\\n'", 'ana', 'como', 'eu', 'nome', 'às', "'vamos", "prazer\\n'", "competição\\n'"]


In [13]:
# Inicializa o dicionário de palavras com os tokens especiais do BERT
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

In [14]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


In [15]:
# Incluímos as palavras no dicionário e criamos índices
for i, w in enumerate(word_list):
    word_dict[w] = i + 4

In [16]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "'pode": 4, 'uma': 5, "então'": 6, 'abriu': 7, 'nos': 8, 'também': 9, "ser\\n'": 10, 'banana': 11, 'está': 12, 'time': 13, 'conhecer': 14, "celebrar\\n'": 15, 'restaurante': 16, 'sim': 17, "'prazer": 18, "'ótimo": 19, 'sou': 20, 'meu': 21, 'de': 22, 'pizza': 23, "'claro": 24, 'recomenda': 25, "'sim": 26, 'é': 27, "'uau": 28, 'sete': 29, 'futebol': 30, 'tarde': 31, 'em': 32, "hoje\\n'": 33, 'da': 34, 'venceu': 35, 'vemos': 36, "'ok": 37, "ana\\n'": 38, 'no': 39, 'muito': 40, 'comer': 41, 'mais': 42, 'vai': 43, 'novo': 44, 'que': 45, 'carlos': 46, 'a': 47, 'um': 48, 'encontramos': 49, 'você': 50, 'algum': 51, 'parabéns': 52, 'e': 53, "'olá": 54, 'para': 55, 'dizem': 56, 'noite': 57, 'pode': 58, "carlos\\n'": 59, "'obrigado": 60, "fenomenal\\n'": 61, 'ana': 62, 'como': 63, 'eu': 64, 'nome': 65, 'às': 66, "'vamos": 67, "prazer\\n'": 68, "competição\\n'": 69}


In [17]:
# Invertemos a ordem e colocamos os índices como chave e as palavras como valor no dicionário
number_dict = {i: w for i, w in enumerate(word_dict)}

In [18]:
number_dict

{0: '[PAD]',
 1: '[CLS]',
 2: '[SEP]',
 3: '[MASK]',
 4: "'pode",
 5: 'uma',
 6: "então'",
 7: 'abriu',
 8: 'nos',
 9: 'também',
 10: "ser\\n'",
 11: 'banana',
 12: 'está',
 13: 'time',
 14: 'conhecer',
 15: "celebrar\\n'",
 16: 'restaurante',
 17: 'sim',
 18: "'prazer",
 19: "'ótimo",
 20: 'sou',
 21: 'meu',
 22: 'de',
 23: 'pizza',
 24: "'claro",
 25: 'recomenda',
 26: "'sim",
 27: 'é',
 28: "'uau",
 29: 'sete',
 30: 'futebol',
 31: 'tarde',
 32: 'em',
 33: "hoje\\n'",
 34: 'da',
 35: 'venceu',
 36: 'vemos',
 37: "'ok",
 38: "ana\\n'",
 39: 'no',
 40: 'muito',
 41: 'comer',
 42: 'mais',
 43: 'vai',
 44: 'novo',
 45: 'que',
 46: 'carlos',
 47: 'a',
 48: 'um',
 49: 'encontramos',
 50: 'você',
 51: 'algum',
 52: 'parabéns',
 53: 'e',
 54: "'olá",
 55: 'para',
 56: 'dizem',
 57: 'noite',
 58: 'pode',
 59: "carlos\\n'",
 60: "'obrigado",
 61: "fenomenal\\n'",
 62: 'ana',
 63: 'como',
 64: 'eu',
 65: 'nome',
 66: 'às',
 67: "'vamos",
 68: "prazer\\n'",
 69: "competição\\n'"}

In [19]:
# Tamanho do vocabulário
vocab_size = len(word_dict)
print(vocab_size)

70


In [20]:
# Criamos uma lista para os tokens
token_list = list()

In [21]:
# Loop pelas sentenças para criar a lista de tokens
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

In [22]:
token_list

[[54, 63, 43, 64, 20, 47, 38],
 [54, 62, 21, 65, 27, 46, 40, 68],
 [18, 32, 14, 50, 9, 63, 50, 12, 33],
 [19, 21, 13, 22, 30, 35, 47, 69],
 [28, 52, 59],
 [60, 38],
 [67, 41, 5, 23, 42, 31, 55, 15],
 [24, 50, 25, 51, 16, 38],
 [26, 7, 48, 16, 44, 53, 56, 45, 47, 23, 22, 11, 27, 61],
 [37, 8, 49, 39, 16, 66, 29, 34, 57, 58, 10],
 [4, 17, 8, 36, 42, 31, 6]]

In [23]:
# Primeira frase
text[0:29]

"'Olá, como vai? Eu sou a Ana."

In [24]:
# Primeira frase no formato de token (o que será usado para treinar o modelo BERT)
token_list[0]

[54, 63, 43, 64, 20, 47, 38]

> Abaixo estão os Hiperparâmetros usados para controlar o treinamento.

In [25]:
# Hiperparâmetros
batch_size = 6
n_segments = 2
dropout = 0.2

# Comprimento máximo
maxlen = 100 

# Número máximo de tokens que serão previstos
max_pred = 7

# Número de camadas 
n_layers = 6 

# Número de cabeças no multi-head attention
n_heads = 12

# Tamanho da embedding
d_model = 768

# Tamanho da dimensão feedforward: 4 * d_model
d_ff = d_model * 4

# Dimensão de K(=Q)V
d_k = d_v = 64 

# Epochs
NUM_EPOCHS = 50

## Criação dos Batches de Dados e Aplicação dos Tokens Especiais

A função make_batch() abaixo cria lotes (batches) de dados para o treinamento do modelo BERT. Ela é responsável por gerar a entrada correta necessária para o treinamento do BERT, que inclui os tokens de entrada, os tokens mascarados, as posições dos tokens mascarados, os IDs de segmentos e um rótulo indicando se a segunda sentença segue imediatamente a primeira. Vamos descrever cada uma das partes da função e usar imagens para facilitar a compreensão.

Inicialização: A função começa inicializando um lote vazio e contadores para sentenças positivas e negativas. Sentenças positivas são pares de sentenças onde a segunda sentença segue imediatamente a primeira, enquanto as negativas são pares onde isso não ocorre. O lote deve ser equilibrado entre sentenças positivas e negativas.

Geração de pares de sentenças: Para cada instância no lote, a função seleciona aleatoriamente duas sentenças do conjunto de dados. Cada sentença é então convertida em uma lista de IDs de tokens e os tokens especiais [CLS] e [SEP] são adicionados nos lugares apropriados.

Segment IDs: Para cada par de sentenças, a função gera IDs de segmentos, que são 0 para tokens na primeira sentença e 1 para tokens na segunda sentença.

Masked Language Model (MLM): A função então seleciona aleatoriamente 15% dos tokens para mascarar para a tarefa de MLM, garantindo que os tokens [CLS] e [SEP] não sejam mascarados. Esses tokens são substituídos pelo token [MASK], por um token aleatório ou permanecem inalterados, dependendo de um sorteio aleatório.

![DSA](imagens/bert2.png)

Padding: A função adiciona padding aos IDs de entrada, aos IDs de segmento, aos tokens mascarados e às posições mascaradas para garantir que todas as listas tenham o mesmo comprimento.

Next Sentence Prediction: Por último, a função verifica se a segunda sentença segue imediatamente a primeira. Se sim, ela adiciona um rótulo True à instância e incrementa o contador de positivos. Se não, ela adiciona um rótulo False e incrementa o contador de negativos.

![DSA](imagens/bert3.png)

Esta função continua gerando instâncias até que o lote esteja cheio e contenha uma quantidade igual de instâncias positivas e negativas. Então, o lote é retornado.

Note que esta função é apenas um exemplo de como os dados podem ser preparados para o treinamento do BERT. Dependendo do conjunto de dados e da tarefa específica, pode ser necessário ajustar esta função.

A principal inovação técnica do BERT é aplicar o treinamento bidirecional do Transformer, um modelo de atenção popular, à modelagem de linguagem. Isso contrasta com os esforços anteriores que analisavam uma sequência de texto da esquerda para a direita ou um treinamento combinado da esquerda para a direita e da direita para a esquerda. Os resultados do artigo mostram que um modelo de linguagem que é treinado bidirecionalmente pode ter um senso mais profundo de contexto e fluxo de linguagem do que modelos de linguagem de direção única. No artigo, os pesquisadores detalham uma nova técnica chamada Masked LM (MLM), que permite o treinamento bidirecional em modelos nos quais antes era impossível. Link do artigo do BERT: https://arxiv.org/abs/1810.04805

In [26]:
# Função para criar os batches de dados
def make_batch():
    
    batch = []
    
    positive = negative = 0
    
    while positive != batch_size/2 or negative != batch_size/2:
        
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM (MLM) de 15 % dos tokens em uma sentença
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) 

        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        
        shuffle(cand_maked_pos)
        
        masked_tokens, masked_pos = [], []
        
        for pos in cand_maked_pos[:n_pred]:
            
            masked_pos.append(pos)
            
            masked_tokens.append(input_ids[pos])
            
            if random() < 0.8:  
                input_ids[pos] = word_dict['[MASK]'] 
            elif random() < 0.5:  
                index = randint(0, vocab_size - 1) 
                input_ids[pos] = word_dict[number_dict[index]] 

        # Zero Paddings
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            
            # IsNext
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) 
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            
            # NotNext
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) 
            negative += 1
            
    return batch

A função get_attn_pad_masked() abaixo cria uma máscara de atenção para tokens de padding em uma sequência.

Entradas: A função aceita duas sequências, seq_q e seq_k. Estas são tipicamente a sequência de consulta (query) e a sequência chave (key) em uma operação de atenção.

Extração de tamanho: A função extrai o tamanho do lote (batch_size) e os comprimentos das sequências (len_q e len_k) a partir das dimensões das sequências de entrada.

Criação da máscara: A máscara de atenção é criada verificando quais elementos em seq_k são iguais a zero (o que indica um token de padding). Isso produz uma matriz booleana do mesmo tamanho que seq_k, onde True indica um token de padding e False indica um token real.

Adição de uma dimensão: A dimensão é adicionada à máscara usando o método unsqueeze(1), que adiciona uma dimensão extra no índice 1. Isso é necessário porque a máscara de atenção deve ter a mesma dimensão que as matrizes de atenção no Transformer.

Expansão da máscara: Finalmente, a máscara é expandida para ter o mesmo tamanho que a matriz de atenção, que tem dimensões (batch_size, len_q, len_k). A máscara expandida é retornada pela função.

Em resumo, a função get_attn_pad_masked cria uma máscara que pode ser usada para impedir que o modelo preste atenção aos tokens de padding quando calcula a atenção. Tokens de padding são usados para preencher sequências para que todas tenham o mesmo comprimento, mas eles não carregam nenhuma informação útil, por isso é importante garantir que o modelo os ignore.

In [27]:
# Função para o padding
def get_attn_pad_masked(seq_q, seq_k):
    
    batch_size, len_q = seq_q.size()
    
    batch_size, len_k = seq_k.size()
    
    pad_attn_masked = seq_k.data.eq(0).unsqueeze(1)
    
    return pad_attn_masked.expand(batch_size, len_q, len_k)

In [28]:
# Cria um batch
batch = make_batch()

In [29]:
# Extrai os elementos do batch
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [30]:
# Aplica a função de padding
get_attn_pad_masked(input_ids, input_ids)[0][0], input_ids[0]

(tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
 tensor([ 1, 37,  8, 49, 39, 16,  3, 29, 34, 57, 58, 10,  2,  3, 63, 43, 64, 20,
         47, 38,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  

## Construção do Modelo

Leia o manual em pdf que descreve em detalhes as imagens abaixo.

A imagem abaixo é uma descrição de alto nível do codificador Transformer. A entrada é uma sequência de tokens, que são primeiro incorporados em vetores e depois processados na rede neural. A saída é uma sequência de vetores de tamanho H, em que cada vetor corresponde a um token de entrada com o mesmo índice.

Em termos técnicos, a previsão das palavras de saída requer:

- 1- Adicionar uma camada de classificação na parte superior da saída do codificador.
- 2- Multiplicar os vetores de saída pela matriz embedding, transformando-os na dimensão do vocabulário.
- 3- Calcular a probabilidade de cada palavra no vocabulário com softmax.

A função de perda no modelo BERT leva em consideração apenas a previsão dos valores mascarados e ignora a previsão das palavras não mascaradas. Como consequência, o modelo converge mais lentamente do que os modelos direcionais, uma característica que é compensada por sua maior percepção do contexto.

![DSA](imagens/bert4.png)

No processo de treinamento BERT, o modelo recebe pares de sentenças como entrada e aprende a prever se a segunda sentença do par é a sentença subsequente no documento original. Durante o treinamento, 50% das entradas são um par em que a segunda sentença é a sentença subsequente no documento original, enquanto nos outros 50% uma sentença aleatória do corpus é escolhida como a segunda sentença. 

Para ajudar o modelo a distinguir entre as duas sentenças em treinamento, a entrada é processada da seguinte maneira antes de entrar no modelo:

- 1- Um token [CLS] é inserido no início da primeira frase e um token [SEP] é inserido no final de cada frase.
- 2- Uma embedding de frase indicando a Sentença A ou a Sentença B é adicionada a cada token. As embeddings de sentença são semelhantes em conceito às embeddings de token com um vocabulário de 2.
- 3- Uma embedding posicional é adicionada a cada token para indicar sua posição na sequência. O conceito e a implementação da embedding posicional são apresentados no artigo Transformer.

De fato a embedding usada para treinar o modelo é uma combinação de várias embeddings.

![DSA](imagens/bert1.png)

In [31]:
# Função de ativação GeLu
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

#### Módulo Embedding

A classe de Embedding abaixo faz parte da arquitetura BERT. Componentes individuais da Classe:

Inicialização (def init(self)): O construtor da classe inicializa os componentes necessários para as embeddings.

self.tok_embed: Esta é a camada de embedding de token que mapeia cada token para um vetor de dimensão d_model.

self.pos_embed: Esta é a camada de embedding de posição que mapeia a posição de um token dentro de uma sequência para um vetor de dimensão d_model.

self.seg_embed: Esta é a camada de embedding de segmento que mapeia o tipo de token (0 para a primeira sentença e 1 para a segunda sentença) para um vetor de dimensão d_model.

self.norm: Este é o componente de normalização da camada que é usado para normalizar os vetores de embedding.

Método Forward (def forward(self, x, seg)): O método forward é onde a embedding real acontece.

- Primeiro, ele calcula a posição de cada token na sequência.
- Em seguida, ele cria uma matriz de posições da mesma forma que a entrada x usando pos.unsqueeze(0).expand_as(x).
- Depois, ele calcula a embedding total como a soma das embeddings de token, posição e segmento.
- Finalmente, ele normaliza a embedding usando a camada de normalização e retorna o resultado.

A combinação dessas três embeddings permite ao BERT levar em consideração tanto o significado individual dos tokens quanto a ordem em que aparecem na sequência, bem como se o token pertence à primeira ou à segunda sentença. Isso torna a embedding do BERT muito poderosa e flexível.

In [32]:
# Classe Embedding
class Embedding(nn.Module):
    
    # Método construtor
    def __init__(self):
        
        super(Embedding, self).__init__()
        
        # Token embedding
        self.tok_embed = nn.Embedding(vocab_size, d_model)  
        
        # Position embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  
        
        # Segment (tipo de token) embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  
        
        # Normalização de camada
        self.norm = nn.LayerNorm(d_model)

    # Método Forward
    def forward(self, x, seg):
        
        seq_len = x.size(1)
        
        pos = torch.arange(seq_len, dtype = torch.long)
        
        # (seq_len,) -> (batch_size, seq_len)
        pos = pos.unsqueeze(0).expand_as(x)  
        
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        
        return self.norm(embedding)

#### Módulo Scaled Dot Product Attention

Abaixo está a implementação do mecanismo de Atenção por Produto Escalar Normalizado (Scaled Dot-Product Attention), que é uma parte chave do modelo Transformer, utilizado no BERT e em outros modelos de processamento de linguagem natural.

Aqui está uma explicação linha a linha do método forward:

Pontuações (Scores): O produto escalar de Q (matriz de consulta) e K (matriz chave) é calculado para determinar a pontuação para cada par de chave-consulta. Essas pontuações determinam o quanto cada elemento da sequência de entrada deve ser atendido na produção da representação de saída para um determinado elemento. A pontuação é então escalada pela raiz quadrada da dimensão das chaves (d_k) para evitar que os valores de produto escalar se tornem muito grandes em ambientes de alta dimensão.

Máscara de Atenção: A máscara de atenção é aplicada às pontuações ao preencher os locais onde a máscara tem valor 1 com um número muito grande negativo (-1e9). Isso garante que esses locais recebam um peso próximo de zero quando a softmax for aplicada.

Softmax: A função softmax é aplicada ao último eixo das pontuações para obter os pesos de atenção. Isso garante que todos os pesos são positivos e somam 1, então eles podem ser interpretados como probabilidades.

Contexto: Os pesos de atenção são então multiplicados pela matriz de valores V (value) para obter a saída do mecanismo de atenção. Cada valor é ponderado pela quantidade que deveríamos "atender" a esse valor, conforme determinado pelos pesos de atenção.

O método retorna o contexto (a saída ponderada) e a matriz de atenção.

No modelo Transformer, a Atenção por Produto Escalar Normalizado é usada várias vezes em cada camada, permitindo que o modelo dê atenção a diferentes partes da entrada ao produzir cada elemento da saída. Isso permite que o Transformer lide efetivamente com as dependências de longo alcance entre as palavras nas sequências de entrada.

In [33]:
# Classe ScaledDotProductAttention
class ScaledDotProductAttention(nn.Module):
    
    def __init__(self):
        
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) 
        
        scores.masked_fill_(attn_mask, -1e9) 
        
        attn = nn.Softmax(dim = -1)(scores)
        
        context = torch.matmul(attn, V)
        
        return context, attn

#### Módulo Multi-Head Attention

Abaixo está a implementação de Atenção Multi-Cabeças (Multi-Head Attention), que é um componente chave da arquitetura Transformer, usada em modelos como BERT. A ideia da atenção multi-cabeças é aplicar a atenção do produto escalar normalizado várias vezes em paralelo, cada uma com diferentes pesos aprendidos. Isso permite ao modelo focar em diferentes posições e capturar vários tipos de informação.

Vamos analisar linha a linha o método forward:

Inicialização: residual e batch_size são inicializados com Q e o tamanho do primeiro eixo de Q, respectivamente. O residual será utilizado mais tarde para o caminho de conexão residual.

Transformações Lineares: Aplicamos transformações lineares aos dados de entrada (Q, K e V) usando pesos diferentes. Essas transformações geram múltiplas "cabeças" de atenção.

Remodelagem: As saídas dessas transformações lineares são então reformuladas e transpostas para terem a forma apropriada para a atenção de produto escalar normalizado.

Máscara de Atenção: A máscara de atenção é ajustada para corresponder ao formato das cabeças de atenção.

Atenção de Produto Escalar Normalizado: A atenção de produto escalar normalizado é então aplicada a cada uma das cabeças de atenção.

Remodelagem do Contexto: A saída (contexto) de cada cabeça de atenção é então reformulada e concatenada.

Transformação Linear e Normalização: Uma transformação linear é aplicada ao contexto concatenado, seguida de uma normalização de camada.

Conexão Residual: O resultado final é obtido somando a saída da normalização de camada ao caminho de conexão residual (entrada original Q).

Finalmente, a função retorna a saída normalizada e a matriz de atenção. A atenção multi-cabeças permite que o modelo considere informações de diferentes partes da sequência de entrada, em diferentes subespaços de representação, ao mesmo tempo, o que melhora a capacidade do modelo de capturar várias características do texto.

In [34]:
# Classe MultiHeadAttention
class MultiHeadAttention(nn.Module):
    
    def __init__(self) -> None:
        
        super(MultiHeadAttention, self).__init__()
        
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)

    def forward(self, Q, K, V, attn_mask):
        
        residual, batch_size = Q, Q.size(0)
        
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        
        context = context.transpose(1,2).contiguous().view(batch_size, -1, n_heads * d_v)
        
        output = nn.Linear(n_heads * d_v, d_model)(context)
        
        return nn.LayerNorm(d_model)(output + residual), attn

In [35]:
# Cria o objeto Embedding
emb = Embedding()

In [36]:
# Gera as Embeddings
embeds = emb(input_ids, segment_ids)

In [37]:
# Gera a máscara de atenção
attenM = get_attn_pad_masked(input_ids, input_ids)

In [38]:
# Gera o MultiHeadAttention
MHA = MultiHeadAttention()(embeds, embeds, embeds, attenM)

In [39]:
# Saída
output, A = MHA

In [40]:
A[0][0]

tensor([[0.0359, 0.0653, 0.0654,  ..., 0.0000, 0.0000, 0.0000],
        [0.0555, 0.0341, 0.0608,  ..., 0.0000, 0.0000, 0.0000],
        [0.0319, 0.0762, 0.0410,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0231, 0.0543, 0.0417,  ..., 0.0000, 0.0000, 0.0000],
        [0.0250, 0.0428, 0.0387,  ..., 0.0000, 0.0000, 0.0000],
        [0.0291, 0.0395, 0.0358,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward0>)

#### Módulo Feedforward Posicional

Esta é a implementação da Rede Feedforward Posicional (PoswiseFeedForward), que é um componente da arquitetura Transformer, utilizada em modelos como o BERT.

A Rede Feedforward Posicional é composta por duas camadas lineares com uma ativação GELU (Gaussian Error Linear Unit) entre elas.

Aqui está a explicação detalhada do método forward:

Primeira Camada Linear (self.fc1): A entrada x é passada por uma camada linear (também chamada de camada totalmente conectada). Essa camada tem uma transformação linear com d_model entradas e d_ff saídas, onde d_model é a dimensão do espaço de incorporação e d_ff é a dimensão da camada oculta da rede feed-forward. Isso permite ao modelo aprender representações não-lineares.

Ativação GELU: Em seguida, a ativação GELU é aplicada. A função GELU permite ao modelo aprender transformações mais complexas e não lineares. Ela ajuda a lidar com o problema do desaparecimento do gradiente, permitindo que mais informações passem através da rede.

Segunda Camada Linear (self.fc2): Por fim, a saída da ativação GELU é passada por uma segunda camada linear, que transforma a saída de volta para a dimensão original d_model. Isso é feito para que a saída desta rede feedforward possa ser somada à entrada original (conexão residual) no Transformer.

O retorno da função é, portanto, a saída dessa segunda camada linear, que passou pela transformação da primeira camada linear, ativação GELU, e segunda camada linear.

As redes feed-forward posicionais são uma parte importante dos modelos Transformer, permitindo-lhes aprender representações mais complexas e fazer transformações não-lineares dos dados de entrada.

In [41]:
# Classe PoswiseFeedForward
class PoswiseFeedForward(nn.Module):
    
    def __init__(self) -> None:
        
        super(PoswiseFeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, d_ff)
        
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        
        return self.fc2(gelu(self.fc1(x)))

#### Módulo Encoder Layer

Esta classe define uma Camada de Codificador (EncoderLayer), que é um componente da arquitetura Transformer e também é usado em modelos como BERT. Cada camada de codificador no Transformer contém duas subcamadas: uma camada de Atenção Multi-Cabeças e uma Rede Feed-Forward Posicional.

Aqui está a explicação detalhada do método forward:

Atenção Multi-Cabeças (self.enc_self_attn): A entrada enc_inputs passa por uma camada de Atenção Multi-Cabeças, que é usada para que cada palavra na entrada tenha atenção direcionada a todas as outras palavras. Essa camada também recebe uma máscara (enc_self_attn_mask), que é usada para evitar que o modelo preste atenção a certas palavras (como as de preenchimento). A saída da Atenção Multi-Cabeças é outra sequência de representações vetoriais, com a mesma dimensão da entrada. A matriz de atenção que mostra como cada palavra se atentou a todas as outras também é retornada.

Rede Feed-Forward Posicional (self.pos_ffn): A saída da camada de Atenção Multi-Cabeças passa então por uma Rede Feed-Forward Posicional. Esta é uma rede neural simples que opera independentemente em cada posição da sequência (ou seja, a mesma rede é aplicada a cada posição). Isso permite ao modelo aprender representações mais complexas e realizar transformações não-lineares dos dados.

A função retorna a saída desta camada de codificador, que é a saída da Rede Feed-Forward Posicional, junto com a matriz de atenção. Portanto, a entrada e a saída desta camada do codificador têm a mesma dimensão, o que permite que várias dessas camadas de codificador sejam empilhadas para formar o codificador completo do Transformer.

In [42]:
# Classe EncoderLayer
class EncoderLayer(nn.Module):
    
    def __init__(self) -> None:
        
        super(EncoderLayer, self).__init__()
        
        self.enc_self_attn = MultiHeadAttention()
        
        self.pos_ffn = PoswiseFeedForward()

    def forward(self, enc_inputs, enc_self_attn_mask):
        
        enc_inputs, atnn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        
        enc_inputs = self.pos_ffn(enc_inputs)
        
        return enc_inputs, atnn

#### Modelo BERT

Esta classe define o modelo BERT (Bidirectional Encoder Representations from Transformers), um modelo de linguagem de última geração que usa transformers e atenção bidirecional para entender a semântica das palavras dentro de um contexto.

Vamos analisar em detalhes o método forward:

Embedding (self.embedding): Transforma as entradas (input_ids e segment_ids) em vetores densos (embeddings).

Máscara de Atenção (get_attn_pad_masked): Gera uma máscara de atenção para ignorar os tokens de preenchimento (pad) nas entradas.

Camadas de Codificação (self.layers): Passa a saída do embedding e a máscara de atenção através de várias camadas do codificador. Cada camada de codificador é composta por uma camada de atenção multi-cabeças e uma rede feed-forward posicional.

Pooling (self.activ1(self.fc(output[:, 0]))): Aplica uma camada totalmente conectada e uma ativação tangente hiperbólica à primeira posição (o token de classificação) de cada sequência na saída do codificador. Isso resulta em um vetor de representação de sequência.

Classificador (self.classifier): Uma camada totalmente conectada que gera os logits para a tarefa de classificação de próxima sentença.

Extração de Tokens Mascarados (torch.gather(output, 1, masked_pos)): Selecione os vetores de saída correspondentes aos tokens mascarados.

Transformação dos Tokens Mascarados (self.norm(self.activ2(self.linear(h_masked)))): Aplica uma transformação linear, uma ativação GELU e normalização à saída dos tokens mascarados.

Decoder (self.decoder): Uma camada linear que gera os logits para a tarefa de modelagem de linguagem mascarada. Usa os mesmos pesos que a camada de embedding de tokens para a consistência no espaço de representação. Esta função decoder é usada somente para gerar os logits finais e não é usada no processo de aprendizado do modelo.

O método retorna os logits para a tarefa de modelagem de linguagem mascarada e a tarefa de classificação de próxima sentença. Esses logits podem então ser usados para calcular as perdas para ambas as tarefas durante o treinamento.

In [43]:
# Modelo BERT
class BERT(nn.Module):
    
    def __init__(self) -> None:
        
        super(BERT, self).__init__()
        
        self.embedding = Embedding()
        
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        
        self.fc = nn.Linear(d_model, d_model)
        
        self.activ1 = nn.Tanh()
        
        self.linear = nn.Linear(d_model, d_model)
        
        self.activ2 = gelu
        
        self.norm = nn.LayerNorm(d_model)
        
        self.classifier = nn.Linear(d_model, 2)
        
        embed_weight = self.embedding.tok_embed.weight
        
        n_vocab, n_dim = embed_weight.size()
        
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        
        self.decoder.weight = embed_weight
        
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        
        output = self.embedding(input_ids, segment_ids)
        
        enc_self_attn_mask = get_attn_pad_masked(input_ids, input_ids)
        
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        
        h_pooled = self.activ1(self.fc(output[:, 0]))
        
        logits_clsf = self.classifier(h_pooled)
        
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))
        
        h_masked = torch.gather(output, 1, masked_pos)
        
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        
        logits_lm = self.decoder(h_masked) + self.decoder_bias
        
        return logits_lm, logits_clsf

## Treinamento e Avaliação do Modelo

In [44]:
# Cria o modelo
model = BERT()

In [45]:
# Função de erro
criterion = nn.CrossEntropyLoss()

In [46]:
# Otimizador
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [47]:
batch = make_batch()

In [48]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

Abaixo está o ciclo típico de treinamento de uma época em um modelo de aprendizado de máquina. Vamos quebrá-lo em etapas:

optimizer.zero_grad(): Zera os gradientes de todas as variáveis otimizadas. Isso é feito porque os gradientes no PyTorch são acumulados, ou seja, cada vez que chamamos .backward(), os gradientes são somados em vez de substituídos. Então, precisamos limpar esses gradientes acumulados antes de cada passo de otimização.

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos): Alimenta os dados de entrada no modelo e obtém a saída do modelo. A saída é composta de logits_lm e logits_clsf, que são os resultados brutos não normalizados para a tarefa de modelagem de linguagem e a tarefa de classificação, respectivamente.

loss_lm = criterion(logits_lm.transpose(1,2), masked_tokens): Calcula a perda da tarefa de modelagem de linguagem mascarada. criterion é a função de perda, logits_lm.transpose(1,2) são as previsões do modelo e masked_tokens são os alvos verdadeiros.

loss_lm = (loss_lm.float()).mean(): Converte a perda em um tipo de dados de ponto flutuante (se ainda não for) e, em seguida, calcula a média da perda.

loss_clsf = criterion(logits_clsf, isNext): Calcula a perda da tarefa de classificação da próxima frase.

loss = loss_lm + loss_clsf: Combina as duas perdas em uma única perda escalar.

loss.backward(): Calcula os gradientes de todas as variáveis otimizadas. Os gradientes são computados com respeito à perda.

optimizer.step(): Atualiza os parâmetros do modelo usando os gradientes calculados.

Essas etapas são repetidas para cada época de treinamento. Cada época é um ciclo completo através do conjunto de treinamento. Portanto, se NUM_EPOCHS é 10, então o processo de treinamento completo é executado 10 vezes.

In [49]:
# Loop de treino
for epoch in range(NUM_EPOCHS):
    
    optimizer.zero_grad()
    
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    
    loss_lm = criterion(logits_lm.transpose(1,2), masked_tokens)
    
    loss_lm = (loss_lm.float()).mean()
    
    loss_clsf = criterion(logits_clsf, isNext)
    
    loss = loss_lm + loss_clsf
    
    print(f'Epoch: {epoch + 1} | Loss {loss:.4f}')
    
    loss.backward()
    
    optimizer.step()

Epoch: 1 | Loss 67.1648
Epoch: 2 | Loss 83.9593
Epoch: 3 | Loss 296.9586
Epoch: 4 | Loss 133.0423
Epoch: 5 | Loss 53.5106
Epoch: 6 | Loss 34.0669
Epoch: 7 | Loss 31.3566
Epoch: 8 | Loss 19.9872
Epoch: 9 | Loss 35.3451
Epoch: 10 | Loss 38.6164
Epoch: 11 | Loss 18.6592
Epoch: 12 | Loss 21.3668
Epoch: 13 | Loss 26.1293
Epoch: 14 | Loss 28.1905
Epoch: 15 | Loss 28.2236
Epoch: 16 | Loss 25.7394
Epoch: 17 | Loss 25.0417
Epoch: 18 | Loss 22.9130
Epoch: 19 | Loss 20.8714
Epoch: 20 | Loss 24.3999
Epoch: 21 | Loss 19.4341
Epoch: 22 | Loss 18.4059
Epoch: 23 | Loss 18.5944
Epoch: 24 | Loss 19.4257
Epoch: 25 | Loss 18.0532
Epoch: 26 | Loss 18.6622
Epoch: 27 | Loss 19.3121
Epoch: 28 | Loss 17.8534
Epoch: 29 | Loss 17.9634
Epoch: 30 | Loss 21.3748
Epoch: 31 | Loss 18.0867
Epoch: 32 | Loss 16.7785
Epoch: 33 | Loss 18.7112
Epoch: 34 | Loss 18.5004
Epoch: 35 | Loss 17.7398
Epoch: 36 | Loss 18.4515
Epoch: 37 | Loss 17.5236
Epoch: 38 | Loss 16.7070
Epoch: 39 | Loss 16.5959
Epoch: 40 | Loss 14.8597
Epoch: 

## Extraindo as Previsões do Modelo Treinado

In [50]:
# Extrai o batch
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

'Olá, como vai? Eu sou a Ana.\n'
'Olá, Ana, meu nome é Carlos. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau Parabéns, Carlos!\n'
'Obrigado Ana.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Ana?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'
['[CLS]', "'olá", '[MASK]', 'vai', 'eu', 'sou', 'a', "ana\\n'", '[SEP]', "'ok", 'nos', 'encontramos', 'no', 'restaurante', 'às', 'sete', '[MASK]', 'noite', 'pode', 'conhecer', '[SEP]']


In [51]:
# Extrai as previsões dos tokens
logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('Lista de Masked Tokens Reais: ', [pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('Lista de Masked Tokens Previstos: ', [pos for pos in logits_lm if pos != 0])

Lista de Masked Tokens Reais:  [10, 63, 34]
Lista de Masked Tokens Previstos:  [59, 59]


In [52]:
# Extrai as previsões do próximo token
logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext (Valor Real): ', True if isNext else False)
print('isNext (Valor Previsto): ', True if logits_clsf else False)

isNext (Valor Real):  False
isNext (Valor Previsto):  False


# Fim